In [1]:
from collections import Counter
from math import sqrt
from random import Random
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import re
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import svds
#from scipy.stats.stats import spearmanr
from scipy.stats import spearmanr
from random import shuffle
import math
import random
from numpy import *
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
#from sparsesvd import sparsesvd
from numpy.linalg import matrix_rank
from scipy.stats import chi2_contingency
from matplotlib.pyplot import *

In [2]:
def read_vocab(corpus_file, thr):
    vocab = Counter()
    for line in corpus_file:
        vocab.update(Counter(line.strip().split()))
    vocab_pre = dict([(token, count) for token, count in vocab.items() if count >= thr])
    return {token: (i, count) for i, (token, count) in enumerate(vocab_pre.items())}

def build_cooccur(vocab, corpus_pre, win = 2):
    
    cooccur =  lil_matrix((len(vocab), len(vocab)), dtype=np.float64)
    
    for line in corpus_pre:
        tokens = [t for t in line.strip().split() if t in vocab]
        len_tokens = len(tokens)
        #print(tokens)
        
        for i, tok in enumerate(tokens):
            #print(i)
            #print(tok)
            
            start = max(0, i - win)
            end = min(len_tokens, i + win + 1)
            for j in range(start, end):
                if j != i:
                    distance = np.absolute(j-i)
                    increment = 1.0 / float(distance)
                    #increment = 1.0
                    cooccur[vocab[tok][0], vocab[tokens[j]][0]] += increment
    return cooccur

In [3]:
#create the word-context matrix of text8
corpus_file = "D:\\carme\\text8\\text8\\text8"

corpus = []
with open(corpus_file) as f:
    for line in f:
        corpus.append(line)


thr = 100
win = 2
vocab = read_vocab(corpus, thr)
cooccur_sparse = build_cooccur(vocab, corpus, win)

def check_symmetric(a, tol=1e-8):
    return np.all(np.abs(a-a.T) < tol)
cooccur_dense = cooccur_sparse.todense()
check_symmetric(cooccur_dense)


text8_rank = matrix_rank(cooccur_dense)
text8_rank
np.save("D:\\carme\\text8\\decomposed matrix\\cooccur_sparse", cooccur_sparse)
np.save("D:\\carme\\text8\\decomposed matrix\\vocab", vocab)
np.save("D:\\carme\\text8\\decomposed matrix\\cooccur_dense", cooccur_dense)

In [4]:
print(text8_rank)

11815


In [5]:
print(check_symmetric(cooccur_dense))

True


In [6]:
cooccur_dense

matrix([[2.100e+01, 1.000e+00, 2.200e+01, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [1.000e+00, 0.000e+00, 5.900e+01, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [2.200e+01, 5.900e+01, 9.282e+03, ..., 4.000e+00, 0.000e+00,
         2.500e+00],
        ...,
        [0.000e+00, 0.000e+00, 4.000e+00, ..., 3.000e+00, 0.000e+00,
         0.000e+00],
        [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 8.900e+01,
         0.000e+00],
        [0.000e+00, 0.000e+00, 2.500e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00]])

In [7]:
cooccur_sparse

<11815x11815 sparse matrix of type '<class 'numpy.float64'>'
	with 8154924 stored elements in List of Lists format>